# Participación en empresas, sociedades o asociaciones con posible conflicto de interés (4_2_04)
| **Escenario**                                                                    | **Condición exacta evaluada**                                                                                                                      | **Resultado**    |
| -------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------- |
| **No hay participaciones registradas**                                           | La lista `participacion.participacion` no existe o viene vacía                                                                                     | ⚪ **NO_APLICA**  |
| **Faltan datos esenciales**                                                      | No existe: <br>• nombre del ente público, o <br>• sector productivo de la empresa, o <br>• tipo de participación (declarante, pareja, dependiente) | ⚪ **SIN_DATO**   |
| **La empresa/organización pertenece a sector de riesgo T, D o C**                | `sector ∈ sectores_riesgo[funcion_principal_del_ente]`                                                                                             | 🔴 **NO_CUMPLE** |
| **La empresa NO pertenece a sectores de riesgo**                                 | El sector **no** está dentro de T/D/C de la función principal del ente público                                                                     | 🟢 **CUMPLE**    |
| **La lista de participaciones existe pero tiene registros vacíos / incompletos** | Sectores o campos faltantes                                                                                                                        | ⚪ **SIN_DATO**   |


In [ ]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME
import unicodedata


METRIC_ID = "4_2_04_PARTICIPACION_SECTOR_RIESGO"


# -------------------------------------------------------------------
# MATRIZ TEMPORAL (reemplazar por catálogo oficial INEGI)
# -------------------------------------------------------------------
# FUNCION PRINCIPAL DEL ENTE -> SECTORES RIESGO (T, D, C)
MATRIZ_SECTORES_FUNCIONES = {
    "SALUD": ["T", "D"],
    "OBRAS PUBLICAS": ["T", "C"],
    "EDUCACION": ["D"],
    "SEGURIDAD": ["T", "C"],
}


# -------------------------------------------------------------------
# AUXILIARES
# -------------------------------------------------------------------
def limpiar(v):
    if not v:
        return None
    v = str(v).strip().upper()
    v = ''.join(c for c in unicodedata.normalize('NFD', v)
                if unicodedata.category(c) != 'Mn')  # sin acentos
    while "  " in v:
        v = v.replace("  ", " ")
    return v


def extraer(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None
        return actual
    except:
        return None


# -------------------------------------------------------------------
# EVALUADOR DE MÉTRICA
# -------------------------------------------------------------------
def evaluar_metrica(doc):

    # A) Nombre del ente público
    ente = limpiar(
        extraer(doc, "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nombreEntePublico")
    )

    if not ente:
        return "SIN_DATO"

    funcion_principal = ente  # temporal, ajustar cuando se tenga catálogo real
    sectores_riesgo = MATRIZ_SECTORES_FUNCIONES.get(funcion_principal, [])

    # B) Participaciones declaradas (declarante, pareja, dependientes)
    participaciones = extraer(
        doc, "declaracion.intereses.participacion.participacion"
    )

    if not participaciones:
        return "NO_APLICA"

    if not isinstance(participaciones, list):
        participaciones = [participaciones]

    hubo_datos = False

    for part in participaciones:

        sector = limpiar(part.get("sector"))
        tipo = limpiar(part.get("tipo"))  # ejemplo "DECLARANTE", "PAREJA", "DEPENDIENTE"

        # Falta de dato relevante
        if not sector:
            return "SIN_DATO"

        hubo_datos = True

        # regla principal: si sector ∈ riesgo → NO_CUMPLE
        if sector in sectores_riesgo:
            return "NO_CUMPLE"

    if not hubo_datos:
        return "NO_APLICA"

    return "CUMPLE"


# -------------------------------------------------------------------
# PROCESADOR MONGO
# -------------------------------------------------------------------
def procesar_metrica_4_2_04():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"\n🚀 Procesando métrica {METRIC_ID}...\n")

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]

        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosEmpleoCargoComision": 1,
            "declaracion.intereses.participacion": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"⚠️ Error en documento {doc.get('_id')}: {e}")

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f" > {total} documentos procesados...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print("Total procesados:", total)
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass

        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_2_04()



🚀 Procesando métrica 4_2_04_PARTICIPACION_SECTOR_RIESGO...



c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


 > 2000 documentos procesados...
 > 4000 documentos procesados...
 > 6000 documentos procesados...
 > 8000 documentos procesados...
 > 10000 documentos procesados...
 > 12000 documentos procesados...
 > 14000 documentos procesados...
